## Это наш лицевой файл, к которому будем подключать свои скрипты. Тут будут основные визуальные выводы. Логика вычислений и обработки будет вынесена в отдельные файлы .py


In [1]:
import sys
import os

cur_dir = os.getcwd()
dataset_path = os.path.join(cur_dir, "../Dataset")
script_path = os.path.join(cur_dir, "../Scripts")
wavelets_path = os.path.join(cur_dir, "../Wavelet")
print(wavelets_path)
sys.path.append(dataset_path)
sys.path.append(script_path)
sys.path.append(wavelets_path)

d:\УЧЕБА\Remote monitoring\MyNN\../Wavelet


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from keras.optimizers import SGD

In [3]:
from importlib import reload
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
import ECGsignal as ecg

reload(ecg)

ecg.init()

d:\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
import CWT as wt

In [6]:
reload(wt)
wt.init()

In [7]:

test_ecg = ecg.signal(number_labels={"VA" : 300, "AF" : 300, "SR" : 300})

wt.cwt_transform_df(test_ecg, 0, 2, wt.wavelet()[3])


,data,label,CWT
0,"[[-0.025, -0.025, -0.025, -0.025, -0.025, -0.0...",VA,"[[[(1.781578525410848+0.0465330249603444j), (0..."
1,"[[-0.015, -0.015, -0.015, -0.015, -0.015, -0.0...",VA,"[[[(2.0676746397196286+0.03439586579231204j), ..."
2,"[[-0.12, -0.12, -0.12, -0.12, -0.12, -0.12, -0...",VA,"[[[(0.41941757943509067-0.03390905844045573j),..."
3,"[[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.044, 0...",VA,"[[[(1.8262632061848432-0.008308847430309628j),..."
4,"[[-0.055, -0.055, -0.055, -0.055, -0.055, -0.0...",VA,"[[[(3.2670964605441224+0.251179389306744j), (1..."
...,...,...,...
895,"[[-0.14, -0.14, -0.14, -0.14, -0.14, -0.14, -0...",SR,"[[[(2.186536399726562+0.3448740159861551j), (0..."
896,"[[-0.32, -0.32, -0.32, -0.321, -0.319, -0.32, ...",SR,"[[[(-0.6970831469901372-0.06308556922682065j),..."
897,"[[-0.02, -0.02, -0.02, -0.02, -0.02, -0.02, -0...",SR,"[[[(3.3794469689340634+0.3628565755703701j), (..."
898,"[[-0.1, -0.1, -0.1, -0.1, -0.099, -0.1, -0.117...",SR,"[[[(2.151739366604096-0.13887586731907928j), (..."


In [115]:
test_ecg["CWT"][1].shape

(2, 96, 1000)

In [8]:
test_ecg = test_ecg.replace({"label" : {"SR":0, "VA":1, "AF":2}})

In [9]:
X_train, X_test, y_train, y_test \
    = train_test_split(np.array(test_ecg["CWT"]), np.array(test_ecg["label"]), test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val \
    = train_test_split(np.array(X_train), np.array(y_train), test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

# train 80%, test 20%, val 20%

In [10]:
X_train = np.stack(X_train, axis=0)
X_val = np.stack(X_val, axis=0)
X_test = np.stack(X_test, axis=0)

In [11]:
# Преобразование меток в one-hot кодировку
# y_train = to_categorical(y_train, num_classes=3).numpy()
# y_val = to_categorical(y_val, num_classes=3)
# y_test = to_categorical(y_test, num_classes=3)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[2], X_train.shape[3], X_train.shape[1]))
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val = X_val.reshape((X_val.shape[0], X_val.shape[2], X_val.shape[3], X_val.shape[1]))
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)

X_test = X_test.reshape((X_test.shape[0], X_test.shape[2], X_test.shape[3], X_test.shape[1]))
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

y_train = tf.one_hot(y_train, depth=3).numpy()
y_val = tf.one_hot(y_val, depth=3).numpy()
y_test = tf.one_hot(y_test, depth=3).numpy()

d:\anaconda3\Lib\site-packages\tensorflow\python\framework\constant_op.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


In [120]:
X_val.shape

TensorShape([180, 96, 1000, 2])

In [111]:


# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape = 5
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape, conv_shape), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 1.0777 - accuracy: 0.3889 - val_loss: 1.1326 - val_accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 1s 1s/step - loss: 1.1517 - accuracy: 0.2222 - val_loss: 1.1309 - val_accuracy: 0.0000e+00
Epoch 3/15
1/1 [==============================] - 1s 1s/step - loss: 1.0313 - accuracy: 0.4444 - val_loss: 1.1241 - val_accuracy: 0.0000e+00
Epoch 4/15
1/1 [==============================] - 1s 1s/step - loss: 1.1061 - accuracy: 0.3889 - val_loss: 1.1262 - val_accuracy: 0.0000e+00
Epoch 5/15
1/1 [==============================] - 1s 1s/step - loss: 1.1011 - accuracy: 0.2778 - val_loss: 1.1253 - val_accuracy: 0.0000e+00
Epoch 6/15
1/1 [==============================] - 1s 1s/step - loss: 0.9790 - accuracy: 0.5000 - val_loss: 1.1238 - val_accuracy: 0.1667
Epoch 7/15
1/1 [==============================] - 1s 1s/step - loss: 1.0597 - accuracy: 0.3889 - val_loss: 1.1245 - val_accuracy: 0.1667
Epoch 8/15
1/1 [=====

# Выше результаты для вейвлета морле с выборкой 300


In [121]:

# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape = 5
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape, conv_shape), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/25
17/17 [==============================] - 32s 2s/step - loss: 1.1115 - accuracy: 0.3981 - val_loss: 1.0417 - val_accuracy: 0.4611
Epoch 2/25
17/17 [==============================] - 34s 2s/step - loss: 1.0380 - accuracy: 0.4519 - val_loss: 0.9973 - val_accuracy: 0.5056
Epoch 3/25
17/17 [==============================] - 32s 2s/step - loss: 0.9787 - accuracy: 0.5241 - val_loss: 0.9723 - val_accuracy: 0.5056
Epoch 4/25
17/17 [==============================] - 32s 2s/step - loss: 0.9631 - accuracy: 0.5259 - val_loss: 0.9471 - val_accuracy: 0.5167
Epoch 5/25
17/17 [==============================] - 46s 3s/step - loss: 0.9270 - accuracy: 0.5444 - val_loss: 0.9278 - val_accuracy: 0.5611
Epoch 6/25
17/17 [==============================] - 44s 3s/step - loss: 0.8948 - accuracy: 0.5611 - val_loss: 0.9095 - val_accuracy: 0.5889
Epoch 7/25
17/17 [==============================] - 43s 3s/step - loss: 0.8836 - accuracy: 0.6111 - val_loss: 0.9026 - val_accuracy: 0.5667
Epoch 8/25
17/17 [==

# Выше результат для выборки в 900 для вейвлета мексики

In [12]:
# Предположим, что X_train, y_train, X_val, y_val, X_test, y_test - это ваши данные
spec_height = test_ecg["CWT"][0].shape[1]
spec_width = test_ecg["CWT"][0].shape[2]
channels = 2  # для двух каналов в спектрограмме
conv_shape = 5
opt = SGD(learning_rate = 0.0001)

# Создание модели
model = models.Sequential()
model.add(layers.Conv2D(32, (conv_shape, conv_shape), activation='relu', input_shape=(spec_height, spec_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (conv_shape, conv_shape), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Компиляция модели
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))

# Оценка производительности на тестовом наборе
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)



Epoch 1/25


17/17 [==============================] - 51s 3s/step - loss: 1.1430 - accuracy: 0.3722 - val_loss: 1.0552 - val_accuracy: 0.5000
Epoch 2/25
17/17 [==============================] - 45s 3s/step - loss: 1.0467 - accuracy: 0.4519 - val_loss: 1.0098 - val_accuracy: 0.4778
Epoch 3/25
17/17 [==============================] - 43s 3s/step - loss: 0.9846 - accuracy: 0.5167 - val_loss: 0.9773 - val_accuracy: 0.5000
Epoch 4/25
17/17 [==============================] - 44s 3s/step - loss: 0.9715 - accuracy: 0.5241 - val_loss: 0.9563 - val_accuracy: 0.5111
Epoch 5/25
17/17 [==============================] - 43s 3s/step - loss: 0.9207 - accuracy: 0.5648 - val_loss: 0.9336 - val_accuracy: 0.5222
Epoch 6/25
17/17 [==============================] - 42s 2s/step - loss: 0.8944 - accuracy: 0.5926 - val_loss: 0.9095 - val_accuracy: 0.5833
Epoch 7/25
17/17 [==============================] - 40s 2s/step - loss: 0.8817 - accuracy: 0.6037 - val_loss: 0.9114 - val_accuracy: 0.5500
Epoch 8/25
17/17